# Torsion Lab Analysis Code ヽ(^o^)丿

## Define Dimension and Material Properties
### YOU WILL NEED TO INPUT THESE

In [ ]:
from math import pi

#Specimen Dimensions
L = []  # Length of Bar [m]
R = []  # Radius of the Bar [m]
J = pi*R**4/2  #Polar 2nd moment of area [m^4]

#Tensile Properties
E = []  # Young's Modulus [Pa]
nu = [] # Poisson's ratio
s_y = [] # Yield Strength [Pa]

#Shear Properties
G_th = E/2/(1+nu)  # Theoretical Shear Modulus [Pa]
tau_y = s_y/(3**0.5)   # Yield Shear Stress [Pa]

## Twist/Force Data
### YOU WILL NEED TO INPUT ANGLE AND FORCE DATA

In [ ]:
import numpy as np

#Raw data from experiments
AngleDeg = np.array([]) #angle in degrees
ForceKg = np.array([]) #force in kgf
Sample = 'Material Type'

#Convert angle and calculate torque
D = 0.0523
Twist = [] #angle in radians
Force = [] #force in Newtons
Torque = [] #torque on the rod

### Plot the Data

In [ ]:
import matplotlib.pyplot as plt #Matlab-esque plotting library

#Plot the experimental torque vs twist angle
plt.figure(figsize = (6,5))
plt.plot(Twist, Torque,'.-',label = Sample)
plt.xlabel('Twist Angle (rad) ',fontsize = 16)
plt.ylabel('Torque (Nm)',fontsize = 16)
plt.title('Torque vs Twist Angle', fontsize = 20)
plt.legend();

## Useful Analysis Functions

In [ ]:
from scipy.stats import linregress #linear regression function built into Scipy 

def shearModulusFit(twist,torque,R,L,a,b):
    '''This is a linear fit to twist and torque data between the indices a and b. 
    Note, this will return an error if a or b are outside the length of the data. 
    R is the rod radius, L is the rod length.
    This function is only valid for use in the elastic region of twist.'''
    
    #Find shear stress and shear strain using elastic relationships
    shearStrain = twist[a:b]*R/L
    shearStress = 2*torque[a:b]/pi/R**3
    
    #Fit the modulus
    #This gives the slope (G), intercept (C), regression (r), P-value and standard error
    G,C,r,P,Err = linregress(shearStrain,shearStress) 
    #Note: Python lets you save multivariable outputs with a comma, 
    #e.g. a,b=[1,2] will give a=1 and b=2
    
    #Make a line for the fit data
    Y = [0.0, max(shearStress)] #this is a list of length 2 for plotting the fit later
    X = [(y-C)/G for y in Y] #these are inverted from y=G*x+C, x=(y-C)/G
    return G,C,r,X,Y

def elasticPlasticTorque(twist,tau_y,H,n,G,L,R):
    '''This is a theoretical calculation of the torque response for a material with
    given material properties after a specified twist. This uses the material shear
    yield strength tau_y, hardening modulus H, hardening exponent n, shear modulus G,
    and the rod length L and radius R.'''
    
    #Calculate individual elastic and plastic torque
    T_Elastic = []
    T_Plastic = []
    for t in twist:
        ry = tau_y*L/G/t #yield radius

        #Elastic Deformation
        if ry > R:
            T_Elastic += [pi*G*t*R**4/2/L]
            T_Plastic += [0]
        else:
            #Plastic + Elastic Deformation
            T_Elastic += [pi*tau_y*ry**3/2]
            T_Plastic += [2*pi*H/3**0.5/(n+3)*(t/3**0.5/L)**n*(R**(n+3)-ry**(n+3))]

    #Combined elastic/plastic torque
    T = [Tel+Tpl for Tel,Tpl in zip(T_Elastic,T_Plastic)]
    return T_Elastic, T_Plastic, T

## Fit the Shear Modulus
### Then compare it with the theoretical shear modulus from the stiffness
### YOU WILL NEED TO EDIT THE 'a' AND 'b' VALUES

In [ ]:
#Fit to the loading data
a = 0 #Starting point for the fit
b = 12 #Ending point for the fit
G,C,r,X,Y = shearModulusFit(Twist,Torque,R,L,a,b)

#Find the max shear stress and strain from elastic analysis
shearStrain = Twist[a:b]*R/L
shearStress = Torque[a:b]*R/J

#Plot the max shear stress and strain and fit
plt.figure(figsize = (6,5))
plt.plot(shearStrain,shearStress,'.-',label = Sample)
plt.plot(X,Y,'--',label = 'Modulus Fit G='+str(round(G*1e-9,2))+'GPa')
plt.xlabel('Max Shear Strain',fontsize = 16)
plt.ylabel('Max Shear Stress (Pa)',fontsize = 16)
plt.legend();

## Compare the Theoretical and Experimental Torque

In [ ]:
#Experimental Hardening Properties
H = [] # Hardening coefficient [Pa]
n = [] # Hardening exponent
#YOU WILL NEED TO INPUT H AND n VALUES AND MODIFY THEM TO FIT THE DATA

#Determine the elastic, plastic and total torque
eTorque, pTorque, thTorque = elasticPlasticTorque(Twist,tau_y,H,n,G_th,L,R)

#Plot the results
fig, axs = plt.subplots(1, 2, figsize=(10,5))
axs[0].plot(Twist, Torque, '.-', label = 'Experiment')
axs[0].plot(Twist, thTorque, label = 'Theory')
axs[0].set_xlabel('Twist Angle(rad) ',fontsize = 16)
axs[0].set_ylabel('Torque(Nm)',fontsize = 16)
axs[0].set_title('Theoretical vs Experimental Torque',fontsize=16)
axs[0].legend();

#See the elastic and plastic torque contributions
axs[1].plot(Twist, eTorque, label = 'Elastic Torque')
axs[1].plot(Twist, pTorque,label = 'Plastic Torque')
axs[1].set_xlabel('Twist Angle(rad) ',fontsize = 16)
axs[1].set_ylabel('Torque(Nm)',fontsize = 16)
axs[1].set_title('Elastic vs Plastic Torque Components',fontsize=16)
axs[1].set_xlim([0,10]); #Zoom to get a closer look at the elastic component
axs[1].legend();